In [72]:
import pandas as pd
import folium

In [73]:
df = pd.read_csv("datasets V2AIX/df_v4.csv")

In [74]:
df_trajet = pd.read_excel("datasets V2AIX/df_trips.xlsx")

In [75]:
# Supprimer la limite d'affichage des colonnes
pd.set_option('display.max_columns', None)

df.head()

,Unnamed: 0,recording_timestamp_nsec,message_header_protocol_version,message_header_message_id,message_header_station_id_value,message_cam_generation_delta_time_value,message_cam_cam_parameters_basic_container_station_type_value,message_cam_cam_parameters_basic_container_reference_position_latitude_value,message_cam_cam_parameters_basic_container_reference_position_longitude_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_confidence_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_minor_confidence_value,message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_orientation_value,message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_value_value,message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_confidence_value,message_cam_cam_parameters_high_frequency_container_choice,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_heading_heading_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_heading_heading_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_speed_speed_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_speed_speed_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_drive_direction_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_length_vehicle_length_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_length_vehicle_length_confidence_indication_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_vehicle_width_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_longitudinal_acceleration_longitudinal_acceleration_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_longitudinal_acceleration_longitudinal_acceleration_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_curvature_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_curvature_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_curvature_calculation_mode_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_yaw_rate_yaw_rate_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_yaw_rate_yaw_rate_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_bits_unused,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_acceleration_control_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_lane_position_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_lane_position_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_steering_wheel_angle_value_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_steering_wheel_angle_confidence_value,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_frequency_steering_wheel_angle_is_present,message_cam_cam_parameters_high_frequency_container_basic_vehicle_container_high_f

In [76]:
# df_sample = df.sample(100)
# df_sample.to_excel("df_sample.xlsx")

In [77]:
df = df.rename(columns={"message_header_station_id_value": "station_id"})

In [78]:
import pandas as pd

def initialize_cpesp(df):
    # S'assurer que les timestamps sont bien au format datetime
    df = df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # Initialiser pseudonym_id_cpesp à partir des trip_id + index local
    df = df.sort_values(["trip_id", "timestamp"]).reset_index(drop=True)

    df["pseudonym_id_cpesp"] = (
        df.groupby("trip_id").cumcount()
        .apply(lambda i: f"cpesp_{i}")
    )
    df["pseudonym_id_cpesp"] = df["trip_id"].astype(str) + "_" + df["pseudonym_id_cpesp"]

    return df


In [79]:
from sklearn.neighbors import BallTree
import numpy as np
from tqdm import tqdm

def compute_density(df, radius_m=200, time_window_s=5):
    df = df.copy()
    df["density"] = 0

    # Convertir les coordonnées en radians
    coords = np.radians(df[["latitude", "longitude"]].values)
    tree = BallTree(coords, metric="haversine")
    radius_rad = radius_m / 6371000  # Rayon terrestre en mètres

    timestamps = df["timestamp"].values.astype("datetime64[s]").astype("int")

    for i in tqdm(range(len(df))):
        idx = tree.query_radius([coords[i]], r=radius_rad)[0]
        neighbors = df.iloc[idx]
        
        time_diffs = np.abs(neighbors["timestamp"].values.astype("datetime64[s]").astype("int") - timestamps[i])
        active_neighbors = neighbors[(time_diffs <= time_window_s) & (neighbors["station_id"] != df.iloc[i]["station_id"])]
        
        df.at[df.index[i], "density"] = active_neighbors["station_id"].nunique()

    return df


In [80]:
import numpy as np

# Coordonnées de Aachen
aachen_lat = 50.7753
aachen_lon = 6.0839
radius_km = 5

# Conversion du rayon en degrés
lat_range = radius_km / 111
lon_range = radius_km / (111 * np.cos(np.radians(aachen_lat)))

# Filtrage des messages CAM autour d’Aachen
df_aachen = df[
    (df["latitude"].between(aachen_lat - lat_range, aachen_lat + lat_range)) &
    (df["longitude"].between(aachen_lon - lon_range, aachen_lon + lon_range))
].copy()

# Calcul surface et densité locale (si needed)
area_aachen_km2 = np.pi * radius_km**2
n_points = len(df_aachen)
print(f"Surface couverte : {area_aachen_km2:.2f} km²")
print(f"{n_points} CAMs dans un rayon de {radius_km} km autour d’Aachen.")


Surface couverte : 78.54 km²
90951 CAMs dans un rayon de 5 km autour d’Aachen.


In [81]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import random

def apply_cpe(df, radius_m=200, time_step_s=30, k=5):
    df = df.copy()
    df = df.sort_values("timestamp").reset_index(drop=True)

    # Convert timestamps to seconds (relative to the start)
    t0 = df["timestamp"].min()
    df["time_sec"] = (df["timestamp"] - t0).dt.total_seconds()

    # Nouvelle colonne : pseudonym_id_cpesp (si pas encore créée)
    if "pseudonym_id_cpesp" not in df.columns:
        df["pseudonym_id_cpesp"] = df["station_id"].astype(str) + "_0"

    # Initialiser compteur de pseudonymes par véhicule
    pid_counter = {sid: 1 for sid in df["station_id"].unique()}

    # Avancer dans le temps par fenêtres glissantes
    for t in range(0, int(df["time_sec"].max()), time_step_s):
        window = df[(df["time_sec"] >= t) & (df["time_sec"] < t + time_step_s)]
        vehicles = window["station_id"].unique()

        # Pour chaque véhicule, chercher ses voisins dans la fenêtre
        for sid in vehicles:
            ego = window[window["station_id"] == sid]
            if ego.empty:
                continue
            lat0, lon0 = ego.iloc[0][["latitude", "longitude"]]

            neighbors = []
            for other_sid in vehicles:
                if other_sid == sid:
                    continue
                other = window[window["station_id"] == other_sid]
                if other.empty:
                    continue
                lat1, lon1 = other.iloc[0][["latitude", "longitude"]]
                d = geodesic((lat0, lon0), (lat1, lon1)).meters
                if d <= radius_m:
                    neighbors.append(other_sid)

            # Si k-1 voisins, activer CPE
            if len(neighbors) >= k - 1:
                group = [sid] + neighbors
                group = list(set(group))[:k]  # Prendre les k premiers si plus

                # Générer nouvelle permutation des pseudonymes
                new_ids = group.copy()
                random.shuffle(new_ids)
                mapping = dict(zip(group, new_ids))

                # Appliquer à la fenêtre actuelle
                mask = df["station_id"].isin(group) & (df["time_sec"].between(t, t + time_step_s))
                for s in group:
                    df.loc[mask & (df["station_id"] == s), "pseudonym_id_cpesp"] = f"{mapping[s]}_{pid_counter[s]}"
                    pid_counter[s] += 1

    return df


In [82]:
import random

def apply_sp(df, vehicles_no_cpe, min_time_s=120, min_distance_m=800):
    df = df.copy()
    df = df.sort_values(["station_id", "timestamp"])

    # Compteur pseudonyme
    pid_counter = {sid: 1 for sid in vehicles_no_cpe}

    # Appliquer SP véhicule par véhicule
    for sid in vehicles_no_cpe:
        vehicle_df = df[df["station_id"] == sid].copy()
        if vehicle_df.empty:
            continue

        last_time = vehicle_df["timestamp"].iloc[0]
        last_point = vehicle_df.iloc[0][["latitude", "longitude"]]
        last_pseudo = f"{sid}_sp_{pid_counter[sid]}"

        for idx, row in vehicle_df.iterrows():
            delta_t = (row["timestamp"] - last_time).total_seconds()
            delta_d = geodesic(last_point, (row["latitude"], row["longitude"])).meters

            # Tirer aléatoirement une stratégie
            strategy = random.choice(["fixed_time", "random_time", "random_distance"])

            if strategy == "fixed_time" and delta_t >= min_time_s:
                pid_counter[sid] += 1
                last_pseudo = f"{sid}_sp_{pid_counter[sid]}"
                last_time = row["timestamp"]
                last_point = (row["latitude"], row["longitude"])

            elif strategy == "random_time" and delta_t >= random.uniform(120, 360):
                pid_counter[sid] += 1
                last_pseudo = f"{sid}_sp_{pid_counter[sid]}"
                last_time = row["timestamp"]
                last_point = (row["latitude"], row["longitude"])

            elif strategy == "random_distance" and delta_d >= random.uniform(800, 1500):
                pid_counter[sid] += 1
                last_pseudo = f"{sid}_sp_{pid_counter[sid]}"
                last_time = row["timestamp"]
                last_point = (row["latitude"], row["longitude"])

            df.at[idx, "pseudonym_id_cpesp"] = last_pseudo

    return df


In [83]:
df = initialize_cpesp(df)  # crée pseudonym_id_cpesp initial par trip_id

In [84]:
print(df.columns.tolist())

['Unnamed: 0', 'recording_timestamp_nsec', 'message_header_protocol_version', 'message_header_message_id', 'station_id', 'message_cam_generation_delta_time_value', 'message_cam_cam_parameters_basic_container_station_type_value', 'message_cam_cam_parameters_basic_container_reference_position_latitude_value', 'message_cam_cam_parameters_basic_container_reference_position_longitude_value', 'message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_confidence_value', 'message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_minor_confidence_value', 'message_cam_cam_parameters_basic_container_reference_position_position_confidence_ellipse_semi_major_orientation_value', 'message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_value_value', 'message_cam_cam_parameters_basic_container_reference_position_altitude_altitude_confidence_value', 'message_cam_cam_parameters_high_frequency_containe

In [85]:
df_aachen = compute_density(df_aachen, radius_m=200, time_window_s=5)  # ajoute colonne 'density'

100%|██████████| 90951/90951 [1:15:49<00:00, 19.99it/s] 


In [86]:
df_aachen.to_csv("df_aachen.csv")

In [ ]:
df = apply_cpe(df, radius_m=200, time_step_s=30, k=5)

In [ ]:
# Extraire identifiants numériques initiaux
df["base_station_id"] = df["pseudonym_id_cpesp"].str.extract(r"^(\d+)_")[0].astype(int)

# Identifier les véhicules qui n’ont qu’un seul pseudonyme après CPE
cpe_stats = df.groupby("station_id")["pseudonym_id_cpesp"].nunique()
vehicles_without_cpe = cpe_stats[cpe_stats == 1].index.tolist()

In [ ]:
df = apply_sp(df, vehicles_without_cpe)

In [ ]:
# df = compute_density(df, radius_m=200, time_window_s=5)  # ajoute colonne 'density'